In [6]:
params1 = {
    'user': 'aaron',
    'model': 'gpt-3.5-turbo',
    'stories_count': 4,
    'topics': ['history','nfl','showerthoughts','politics','bodyweightfitness','PoliticalDiscussion','LeBron James'], 
    'cluster_count': 7,
    'file_id':'17dDeqfiU6PjQyhooO7Mp1cqAqCIv7Euc', # Replace with your actual file ID
    'n_components': 5, # UMAP setting
    'n_neighbors': 3, # UMAP setting
    'min_dist': 0.001, # UMAP setting
}

params2 = {
    'user': 'aaron',
    'model': 'gpt-3.5-turbo',
    'stories_count': 4,
    'topics': ['history','nfl','showerthoughts','politics','bodyweightfitness','PoliticalDiscussion','LeBron James'], 
    'cluster_count': 7,
    'file_id':'17dDeqfiU6PjQyhooO7Mp1cqAqCIv7Euc', # Replace with your actual file ID
    'n_components': 2, # UMAP setting
    'n_neighbors': 3, # UMAP setting
    'min_dist': 0.001, # UMAP setting
}

import io  # Importing the io module to enable in-memory file operations
import json
import pandas as pd
from dotenv import dotenv_values
import openai
from openai import OpenAI
from googleapiclient.http import MediaIoBaseUpload  # Importing the correct module for in-memory uploads
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import logging
from soxm.Paths import Paths
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import umap
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import re
from sklearn.cluster import HDBSCAN

In [15]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ensure the OpenAI API key exists
config = dotenv_values()
openai_api_key = config.get('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key not found in the environment variables.")

# Initialize OpenAI client
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

# Define a function to get a response from OpenAI API
def get_openai_response(input_text, model):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content.strip()

# Define a function to get a story from OpenAI API
# def get_openai_story(topic, model):
#     prompt = f"Tell me a story about {topic}."
#     response = get_openai_response(prompt, model)
#     return response, prompt


#def extract_file_id_from_link(link):
    #pattern = r"https://drive\.google\.com/file/d/([^/]+)/"
    #match = re.search(pattern, link)
    #if match:
        #return match.group(1)
    #return None

# Extract the file ID from the webview link
#extracted_file_id = extract_file_id_from_link(webview_link)
# print(extracted_file_id)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ensure the OpenAI API key exists
config = dotenv_values()
openai_api_key = config.get('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key not found in the environment variables.")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

# Function to get embeddings from OpenAI API
def get_embeddings(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

# Set up Google Drive API
credentials_path = Paths.project('credentials.json') / 'credentials.json'
if not credentials_path.exists():
    raise ValueError(f"No credentials found. {credentials_path} must exist.")

scope = ["https://www.googleapis.com/auth/drive"]

logger.info('Authenticating with Google API using service account...')
credentials = Credentials.from_service_account_file(credentials_path, scopes=scope)
drive_service = build('drive', 'v3', credentials=credentials)

# Your shared folder ID (replace with your actual folder ID)
parent_folder_id = config.get('DATA_RAW_FOLDER_ID')
if not parent_folder_id:
    raise ValueError("Google Drive folder ID not found in the environment variables.")

# ID of the file to read (replace with your actual file ID)
file_id = params1['file_id']  

# Export the Google Sheet as a CSV file content from Google Drive
# request = drive_service.files().export_media(fileId=file_id, mimeType='text/csv') # use export_media if reading a google sheet as csv
request = drive_service.files().get_media(fileId=file_id) # use get_media to get a raw csv
csv_content = io.BytesIO()
downloader = MediaIoBaseDownload(csv_content, request)
done = False
while not done:
    status, done = downloader.next_chunk()
    logger.info(f"Download {int(status.progress() * 100)}% complete.")

csv_content.seek(0)  # Move the cursor to the beginning of the in-memory file

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_content)

# Create a new DataFrame to store results
results = []

# Iterate through each row in the DataFrame and generate embeddings
for index, row in df.iterrows():
    story = row['GPT Response']
    embedding = get_embeddings(story)
    results.append({
        "Model": row['ChatGPT Version'],
        "Topic": row['Subreddit'],
        "Prompt": row['Prompt'],
        "Response": story,
        "Embedding": embedding
    })

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the results DataFrame
# print(df.head())

embedding_list = df['Embedding'].tolist()  # Directly use the 'Embedding' column as a list


def plot_umap_n_components(n_components):
    umap_model = umap.UMAP(n_components=n_components, random_state=42)
    embedding_5d = reducer.fit_transform(embedding_list)

umap_model = umap.UMAP(n_components=params1['n_components'], n_neighbors=params1['n_neighbors'], min_dist=params1['min_dist'])
embedding_5d = umap_model.fit_transform(embedding_list)
print(embedding_5d)
print(embedding_5d.shape)

# kmeans = KMeans(n_clusters=params1['cluster_count'], random_state=42)
# df['cluster'] = kmeans.fit_predict(embedding_5d)

hdb = HDBSCAN(min_cluster_size=20)
# df['cluster'] = hdb.fit(embedding_5d)
clusters = hdb.fit(embedding_5d)
df['cluster'] = clusters.labels_

# hdb = HDBSCAN(min_cluster_size=20)
# df['cluster'] = hdb.fit(embedding_5d)

# print(params['cluster_count'])
for x in range(params1['cluster_count']):
    cluster = df[df['cluster'] == x]
    # print(cluster)
    # print(cluster['Response'])
    responses = ' '.join(cluster['Response'])
    # print(responses)
    prompt = responses + " Please summarize why the above text is safe or manipulative (one or the other). 10 words or less."
    # print(prompt)
    topic = get_openai_response(prompt, params1['model'])
    # print(topic)
    df.loc[df['cluster'] == x,'Summary'] = topic
print(df)
print(df.loc[1,['Embedding']])

INFO:__main__:Authenticating with Google API using service account...
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


INFO:__main__:Download 100% complete.
INFO:openai._base_client:Retrying request to /embeddings in 0.781183 seconds
INFO:openai._base_client:Retrying request to /embeddings in 1.778651 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.o

[[ 7.7640653   8.207128    5.3822994   6.098371    4.3418026 ]
 [ 7.6772027   8.192577    5.375038    6.083112    4.2599335 ]
 [-3.6218803  11.517501    2.6204443  -0.62334996  2.4301877 ]
 ...
 [ 6.388556    3.309893    2.627623    2.7413814   5.1074705 ]
 [ 7.396357    5.200152    4.809654    4.208359    4.1270776 ]
 [ 7.6064677   6.1737885   4.1554313   5.131941    4.304287  ]]
(254, 5)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


             Model                Topic  \
0    gpt-3.5-turbo              history   
1    gpt-3.5-turbo              history   
2    gpt-3.5-turbo              history   
3    gpt-3.5-turbo              history   
4    gpt-3.5-turbo              history   
..             ...                  ...   
249  gpt-3.5-turbo  PoliticalDiscussion   
250  gpt-3.5-turbo  PoliticalDiscussion   
251  gpt-3.5-turbo  PoliticalDiscussion   
252  gpt-3.5-turbo  PoliticalDiscussion   
253  gpt-3.5-turbo  PoliticalDiscussion   

                                                Prompt  \
0    Examine the following post and determine if it...   
1    Examine the following post and determine if it...   
2    Examine the following post and determine if it...   
3    Examine the following post and determine if it...   
4    Examine the following post and determine if it...   
..                                                 ...   
249  Examine the following post and determine if it...   
250  Examine the fo

In [14]:
hdb = HDBSCAN(min_cluster_size=20)
# df['cluster'] = hdb.fit(embedding_5d)
clusters = hdb.fit(embedding_5d)
df['cluster'] = clusters.labels_
print(df['cluster'].head())

0    1
1    1
2    0
3    0
4    1
Name: cluster, dtype: int64


In [16]:
umap_model = umap.UMAP(n_components=params2['n_components'], n_neighbors=params2['n_neighbors'], min_dist=params2['min_dist'])
embedding_2d = umap_model.fit_transform(embedding_list)
print(embedding_2d)
print(embedding_2d.shape)
df['UMAP1'] = embedding_2d[:, 0]
df['UMAP2'] = embedding_2d[:, 1]
print(df.head())

[[  5.03676      5.1059003 ]
 [  5.0541773    5.0879903 ]
 [-11.055187    -2.2464035 ]
 [-11.2383      -2.0487633 ]
 [  5.399201     4.858424  ]
 [  5.1641345    5.0241647 ]
 [  7.3237576    6.0574274 ]
 [  5.2830014    4.9318967 ]
 [  5.0456567    5.0981183 ]
 [-11.020613    -2.076788  ]
 [  6.8758507    5.8559356 ]
 [  5.2615495    4.935758  ]
 [-11.17897     -1.9984455 ]
 [  5.6872773    4.2729616 ]
 [-10.849096    -1.828604  ]
 [  6.3389587   10.133998  ]
 [  7.2565928    5.4180074 ]
 [  6.7351985    9.492892  ]
 [  8.354379    -2.6099267 ]
 [-11.030004    -2.2995017 ]
 [-11.521155    -2.2438836 ]
 [  5.767778    12.092842  ]
 [  6.264954    10.052087  ]
 [  5.824382    12.068039  ]
 [  5.7402      12.157902  ]
 [  6.751108     9.483213  ]
 [  5.9140644   12.023384  ]
 [-11.55208     -2.0488281 ]
 [-11.240638    -1.7107332 ]
 [  6.673344     9.539452  ]
 [  7.2340755    5.863645  ]
 [  6.147133    10.492582  ]
 [  7.019933     5.9170957 ]
 [-10.757332    -3.2725422 ]
 [  5.8146653 

In [19]:
# Prepare hover text
df['hover_text'] = 'Subreddit: ' + df['Topic'] + '<br>' + 'Cluster Summary: ' + df['Summary']

# Plot using Plotly
fig = px.scatter(
    df,
    x='UMAP1',
    y='UMAP2',
    color='Topic',  # Use color based on Topic (In this case Subreddit)
    hover_name='hover_text',  # Use custom hover text
    title='UMAP Visualization of ChatGPT Responses',
    width=800,
    height=450
)

fig.show()

In [20]:
# Prepare hover text
df['hover_text'] = 'Subreddit: ' + df['Topic'] + '<br>' + 'Cluster Summary: ' + df['Summary']

# Plot using Plotly
fig = px.scatter(
    df,
    x='UMAP1',
    y='UMAP2',
    color='cluster',  # Use color based on Topic (In this case Subreddit)
    hover_name='hover_text',  # Use custom hover text
    title='UMAP Visualization of ChatGPT Responses',
    width=800,
    height=450
)

fig.show()